In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from scipy.optimize import minimize, differential_evolution
import os
import json
from susy_qm import calculate_Hamiltonian

In [40]:
potential = 'AHO'
cutoff = 8
shots = None

In [41]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [42]:
num_qubits

4

In [43]:
min_eigenvalue

np.float64(0.03201011000923872)

In [44]:
HP = qml.pauli_decompose(H)

In [45]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
@qml.qnode(dev)
def circuit(times, operator_ham, op_list, trotter_steps=1):

    #qml.BasisState([1]*num_qubits, wires=range(num_qubits)) # + [0]*(num_qubits-1)
    qml.BasisState([1,1,1,0], wires=range(num_qubits))

    for i, (op, _, _) in enumerate(op_list):
        qml.ApproxTimeEvolution(op, time=times[i], n=trotter_steps)

    qml.ApproxTimeEvolution(operator_ham, time=times[-1], n=trotter_steps)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [46]:
def cost_function(times, operator, op_list):

    #t = pnp.tensor(t, requires_grad=True)
    times = pnp.tensor(times, requires_grad=True)
    #grad_fn = qml.grad(circuit)
    #grad = grad_fn(t, operator, op_list)[0]

    #params = pnp.tensor(params, requires_grad=True)
    energy = circuit(times, operator, op_list)
    
    return energy#-abs(grad)

In [47]:
operator_pool = HP
for i in range(num_qubits):
    operator_pool += qml.PauliX(i)
    operator_pool += qml.PauliY(i)
    operator_pool += qml.PauliZ(i)


In [48]:
num_steps = 5
op_list = []

#variables
max_iter = 1000
strategy = "randtobest1bin"
tol = 1e-3
atol = 1e-3
popsize = 20

bounds = [(0.01, 2 * np.pi)]

for i in range(num_steps):

    print(f"step: {i}")

    gradients = []
    terms = []
    times = []

    x0 = [0.1] * (len(op_list) + 1)

    for op in operator_pool.terms()[1]:

        #'''
        res = minimize(
                cost_function,
                x0=x0,
                bounds = [(0.001, 2*np.pi)],
                args=(op, op_list),
                method= "COBYLA",
                options= {'maxiter':10000, 'tol': 1e-8}
            )
        #'''
        '''
        res = differential_evolution(cost_function,
                                    bounds,
                                    args=(op, op_list),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=atol,
                                    strategy=strategy,
                                    popsize=popsize
                                    )
        '''
        gradients.append(res.fun)
        times.append(res.x)

    if i!=0:
        pre_min_grad = min_grad
    min_arg = np.argmin(gradients)
    min_grad = gradients[min_arg]

    print(f"Min grad: {min_grad}")

    if i!=0:
        if abs(pre_min_grad - min_grad) < 1e-8:
            print("gradient converged")
            break

    min_op = operator_pool[min_arg]
    min_t = times[min_arg]
    print(min_op, min_t)

    op_list.append((min_op,min_t,min_grad))

step: 0
Min grad: 0.9375000000000241
42.156748329868826 * (I(0) @ X(1) @ X(2) @ I(3)) [1.57079634]
step: 1
Min grad: 0.9300868044017949
-9.0625 * (I(0) @ I(1) @ Z(2) @ I(3)) [0.11165287 1.09998307]
step: 2
Min grad: 0.9261597950797428
1.4880445444007506 * (Z(0) @ Y(1) @ Y(2) @ I(3)) [0.10857403 1.03718356 0.13509557]
step: 3
Min grad: 0.9261597945164264
gradient converged


In [50]:
hamiltonian_dict = {op: coeff for op, coeff in zip(HP.terms()[1], HP.terms()[0])}
#hamiltonian_dict

In [51]:
op_coeffs = []
for op, t, grad in op_list:
    coeff = hamiltonian_dict.get(op, None)
    op_coeffs.append((op, coeff, t))

In [52]:
op_coeffs

[(42.156748329868826 * (I(0) @ X(1) @ X(2) @ I(3)), None, array([1.57079634])),
 (-9.0625 * (I(0) @ I(1) @ Z(2) @ I(3)),
  None,
  array([0.11165287, 1.09998307])),
 (1.4880445444007506 * (Z(0) @ Y(1) @ Y(2) @ I(3)),
  None,
  array([0.10857403, 1.03718356, 0.13509557]))]

In [53]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def final_circuit(params):

    #qml.BasisState([1]*num_qubits, wires=range(num_qubits))
    qml.BasisState([1,1,1,0], wires=range(num_qubits))

    for i, (op, coeff, t) in enumerate(op_coeffs):
        pauli_string = qml.pauli.pauli_word_to_string(op)
        qml.PauliRot(params[i], pauli_string, wires=op.wires)

    return qml.state()

In [54]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def test(params):

    qml.BasisState([1,1,1,0], wires=range(num_qubits))

    qml.PauliRot(params[0], 'IXXI', wires=[0,1,2,3])
    qml.PauliRot(params[1], 'IIZI', wires=[0,1,2,3])
    qml.PauliRot(params[2], 'IXXZ', wires=[0,1,2,3])

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [55]:
x0 = np.random.uniform(0, 2 * np.pi, size=3)
print(qml.draw(test)(x0))

0: ─╭|Ψ⟩─╭RIXXI(4.86)─╭RIIZI(4.53)─╭RIXXZ(2.88)─┤ ╭<𝓗(M0)>
1: ─├|Ψ⟩─├RIXXI(4.86)─├RIIZI(4.53)─├RIXXZ(2.88)─┤ ├<𝓗(M0)>
2: ─├|Ψ⟩─├RIXXI(4.86)─├RIIZI(4.53)─├RIXXZ(2.88)─┤ ├<𝓗(M0)>
3: ─╰|Ψ⟩─╰RIXXI(4.86)─╰RIIZI(4.53)─╰RIXXZ(2.88)─┤ ╰<𝓗(M0)>

M0 = 
[[  3.4375    +0.j   0.        +0.j   7.15945616+0.j   0.        +0.j
    5.81753814+0.j   0.        +0.j   1.67705098+0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j]
 [  0.        +0.j  14.5625    +0.j   0.        +0.j  24.0356181 +0.j
    0.        +0.j  17.11632992+0.j   0.        +0.j   4.43705984+0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   0.        +0.j]
 [  7.15945616+0.j   0.        +0.j  39.9375    +0.j   0.        +0.j
   56.9411703 +0.j   0.        +0.j  36.7614778 +0.j   0.        +0.j
    0.        +0.j   0.        +0.j   0.        +0.j   

In [37]:
op_coeffs

[(42.156748329868826 * (I(0) @ X(1) @ X(2) @ I(3)), None, array([1.57079634])),
 (-9.0625 * (I(0) @ I(1) @ Z(2) @ I(3)),
  None,
  array([0.11165286, 1.09996672])),
 (-14.146675794569658 * (I(0) @ X(1) @ X(2) @ Z(3)),
  None,
  array([0.02727352, 1.03897517, 0.42099307]))]

In [49]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_coeffs))
print(qml.draw(final_circuit)(x0))

0: ─╭|Ψ⟩─╭RIXXI(4.75)─╭RIIZI(5.12)─╭RIXXZ(2.60)─┤  State
1: ─├|Ψ⟩─├RIXXI(4.75)─├RIIZI(5.12)─├RIXXZ(2.60)─┤  State
2: ─├|Ψ⟩─├RIXXI(4.75)─├RIIZI(5.12)─├RIXXZ(2.60)─┤  State
3: ─╰|Ψ⟩─╰RIXXI(4.75)─╰RIIZI(5.12)─╰RIXXZ(2.60)─┤  State


In [31]:
eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

In [32]:
min_eigenvalue

np.complex128(0.03201011000923872+0j)

In [33]:
def overlap_function(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = final_circuit(params)
    
    
    overlap = np.vdot(min_eigenvector, ansatz_state)
    cost = np.abs(overlap)**2  
    
    #min_eigenvector_prob = np.abs(min_eigenvector)**2
    #ansatz_prob = np.abs(ansatz_state)**2
    #cost = np.sum(np.sqrt(min_eigenvector_prob * ansatz_prob))

    return (1 - cost)

In [34]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_coeffs))
o_iters = 10000
o_tol=1e-8

print("Running for overlap")

overlap_res = minimize(
    overlap_function,
    x0,
    method= "COBYLA",
    options= {'maxiter':o_iters, 'tol': o_tol}
)

Running for overlap


In [35]:
overlap_res.fun

np.float64(0.04695409286857666)

In [768]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def final_circuit():

    qml.BasisState([1]*num_qubits, wires=range(num_qubits))

    for op, coeff, t in op_coeffs:
        theta = coeff * t[0]
        pauli_string = qml.pauli.pauli_word_to_string(op)
        qml.PauliRot(theta, pauli_string, wires=op.wires)

    return qml.state()